<a href="https://colab.research.google.com/github/Kwanikaze/vpandas/blob/master/AE_OHE_8digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

## Generate Data

In [2]:
def generate_data(num=8):
    """ Generate 'num' number of one-hot encoded integers. """ 
    x_train = np.eye(num)[np.arange(num)]                       # This is a simple way to one-hot encode integers
    
    # Repeat x_train multiple times for training
    x_train = np.repeat(x_train, 100, axis=0)
    
    # The target is x_train itself!
    x_target = x_train.copy()
    return x_train, x_target

In [3]:
num = 8
np.random.seed(10)
x_train, x_target = generate_data(num=num)

In [4]:
print(x_train)
print(np.shape(x_train))
print(np.shape(x_target))

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(800, 8)
(800, 8)


## Autoencoder Parameters

In [5]:
#Parameters
latent_dims = 3
num_epochs = 2000
batch_size = 64
learning_rate = 1e-3
use_gpu = True

## Autoencoder Definition
https://medium.com/pytorch/implementing-an-autoencoder-in-pytorch-19baa22647d1

https://gist.github.com/AFAgarap/4f8a8d8edf352271fa06d85ba0361f26

In [6]:
class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super().__init__()
        #self.encoder_hidden_layer = nn.Linear(in_features=kwargs["input_shape"],)
        self.encoder_layer = nn.Linear(in_features=num, out_features = latent_dims)
        nn.init.xavier_normal_(self.encoder_layer.weight)
        self.decoder_layer = nn.Linear(in_features = latent_dims, out_features = num)
        nn.init.xavier_normal_(self.decoder_layer.weight)
    
    def forward(self,features,latent_dims):
        x_input = self.encoder_layer(features)
        z = torch.sigmoid(x_input)
        if z.size()[0] == latent_dims: #resize from [3] to [1,3]
            z = z.view(1, latent_dims)
        #print(self.decoder_layer(z))
        #recon = torch.sigmoid(self.decoder_layer(z))
        softmax = nn.Softmax(dim=1)
        recon = softmax(self.decoder_layer(z))
        return recon

## Train Autoencoder

In [7]:
def trainAE(AE):
  x_train, x_target = generate_data(num=num)
  inds = list(range(x_train.shape[0]))
  N = x_train.shape[0] # 800
  freq = num_epochs // 10 # floor division

  loss_hist = []
  x_train = Variable(torch.from_numpy(x_train))
  x_target = Variable(torch.from_numpy(x_target))
  for epoch in range(num_epochs):
      inds = np.random.permutation(inds)
      x_train = x_train[inds]
      x_train = x_train.to(device)
      x_target = x_target[inds]
      x_target = x_target.to(device)
      
      loss = 0
      num_batches = N / batch_size
      for b in range(0, N, batch_size):
          #get the mini-batch
          x_batch = x_train[b: b+batch_size]
          x_target_batch = x_target[b: b+batch_size]
          
          #feed forward
          batch_recon = AE(features=x_batch.float(),latent_dims=3)
          
          # Error
          #Convert x_batch from OHE vectors to single scalar for target class, of each sample in batch 
          _, x_batch_targets = x_batch.max(dim=1)
          train_loss = criterion(batch_recon, x_batch_targets)
          #print(batch_recon.size())
          #print(x_batch_targets.size())
          loss += train_loss.item() / N # update epoch loss
          
          #Backprop the error, compute the gradient
          optimizer.zero_grad()
          train_loss.backward()
          
          #update parameters based on gradient
          optimizer.step()
          
      #Record loss per epoch        
      loss_hist.append(loss)
      
      if epoch % freq == 0:
          print()
          print("Epoch %d/%d\tloss=%.5f" % (epoch + 1, num_epochs, loss), end='\t', flush=True)
          
          #Test with all training data
          train_recon = AE(features = x_train.float(),latent_dims=3)
          _, x_targets = x_target.max(dim=1)
          l = criterion(train_recon, x_targets)
          print("Test loss: {:.5f}".format(l.item()), end='')
      
  print("\nTraining finished!")

## Latent dimensions set to 3

In [8]:
#  use gpu if available
device = torch.device("cuda:0" if use_gpu and torch.cuda.is_available() else "cpu")
AE = Autoencoder(latent_dims=3)
AE = AE.to(device)
num_params = sum(p.numel() for p in AE.parameters() if p.requires_grad)
print(AE.parameters)
print("Number of parameters: %d" % num_params) #8*3 + 3 = 27, 3*8 + 8 = 32, 27+32

# optimizer object
optimizer = torch.optim.Adam(params = AE.parameters(), lr = learning_rate)
#criterion = nn.CrossEntropyLoss()    # for target, does not accept a OHE vector
criterion = nn.NLLLoss()

trainAE(AE)

<bound method Module.parameters of Autoencoder(
  (encoder_layer): Linear(in_features=8, out_features=3, bias=True)
  (decoder_layer): Linear(in_features=3, out_features=8, bias=True)
)>
Number of parameters: 59

Epoch 1/2000	loss=-0.00202	Test loss: -0.12471
Epoch 201/2000	loss=-0.00836	Test loss: -0.51502
Epoch 401/2000	loss=-0.01257	Test loss: -0.77480
Epoch 601/2000	loss=-0.01526	Test loss: -0.93856
Epoch 801/2000	loss=-0.01596	Test loss: -0.98204
Epoch 1001/2000	loss=-0.01615	Test loss: -0.99372
Epoch 1201/2000	loss=-0.01621	Test loss: -0.99763
Epoch 1401/2000	loss=-0.01624	Test loss: -0.99909
Epoch 1601/2000	loss=-0.01624	Test loss: -0.99964
Epoch 1801/2000	loss=-0.01625	Test loss: -0.99986
Training finished!


In [9]:
print("Print prediction results:")
x_test = np.eye(num)[np.arange(num)]                        # Test data (one-hot encoded)
x_test = Variable(torch.from_numpy(x_test))
x_test = x_test.to(device)
#np.set_printoptions(2)
for x in x_test:
    print("\tInput: {} \t Output: {}".format(x.cpu().detach().numpy(), np.round(AE(features=x.float(),latent_dims=3).cpu().detach().numpy(),decimals=2)))

Print prediction results:
	Input: [1. 0. 0. 0. 0. 0. 0. 0.] 	 Output: [[1. 0. 0. 0. 0. 0. 0. 0.]]
	Input: [0. 1. 0. 0. 0. 0. 0. 0.] 	 Output: [[0. 1. 0. 0. 0. 0. 0. 0.]]
	Input: [0. 0. 1. 0. 0. 0. 0. 0.] 	 Output: [[0. 0. 1. 0. 0. 0. 0. 0.]]
	Input: [0. 0. 0. 1. 0. 0. 0. 0.] 	 Output: [[0. 0. 0. 1. 0. 0. 0. 0.]]
	Input: [0. 0. 0. 0. 1. 0. 0. 0.] 	 Output: [[0. 0. 0. 0. 1. 0. 0. 0.]]
	Input: [0. 0. 0. 0. 0. 1. 0. 0.] 	 Output: [[0. 0. 0. 0. 0. 1. 0. 0.]]
	Input: [0. 0. 0. 0. 0. 0. 1. 0.] 	 Output: [[0. 0. 0. 0. 0. 0. 1. 0.]]
	Input: [0. 0. 0. 0. 0. 0. 0. 1.] 	 Output: [[0. 0. 0. 0. 0. 0. 0. 1.]]


## Extract intermediate features using Forward Hook

In [10]:
def printnorm_encoder(self, input1, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
    print('\tInside ' + self.__class__.__name__ + ' forward')
    #print('\t input:', input1.cpu().detach().numpy())
    print('\t output rounded to 2 decimals:', np.round(output.cpu().detach().numpy(),decimals=2))
    print('\t output rounded to integer:', np.round(output.cpu().detach().numpy(),decimals=0))

In [11]:
def inside_decoder(self, input1, output):
    # input is a tuple of packed inputs
    # output is a Tensor. output.data is the Tensor we are interested
    print('\tInside ' + self.__class__.__name__ + ' forward')
    #print(input1[0].cpu().detach().numpy())
    #print('\t input:', input1.cpu().detach().numpy())
    print('\t output:', input1[0].cpu().detach().numpy())
    print('\t output rounded to 2 decimals:', np.round(input1[0].cpu().detach().numpy(),2))

In [12]:
#AE.encoder_layer.register_forward_hook(printnorm_encoder)
decoder_hook = AE.decoder_layer.register_forward_hook(inside_decoder)
for x in x_test:
    print('INPUT: {}'.format(x.cpu().detach().numpy()))
    out = AE(features=x.float(),latent_dims=3)
    #print(out)
decoder_hook.remove() #remove hook after use

INPUT: [1. 0. 0. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[2.1524145e-04 9.9979168e-01 3.1583323e-04]]
	 output rounded to 2 decimals: [[0. 1. 0.]]
INPUT: [0. 1. 0. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[0.99932003 0.9995882  0.9997482 ]]
	 output rounded to 2 decimals: [[1. 1. 1.]]
INPUT: [0. 0. 1. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[9.997141e-01 2.497525e-04 9.996780e-01]]
	 output rounded to 2 decimals: [[1. 0. 1.]]
INPUT: [0. 0. 0. 1. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[3.0186510e-04 1.3912453e-04 9.9943691e-01]]
	 output rounded to 2 decimals: [[0. 0. 1.]]
INPUT: [0. 0. 0. 0. 1. 0. 0. 0.]
	Inside Linear forward
	 output: [[9.9956721e-01 9.9946111e-01 1.9269563e-04]]
	 output rounded to 2 decimals: [[1. 1. 0.]]
INPUT: [0. 0. 0. 0. 0. 1. 0. 0.]
	Inside Linear forward
	 output: [[0.00031335 0.0001776  0.00017196]]
	 output rounded to 2 decimals: [[0. 0. 0.]]
INPUT: [0. 0. 0. 0. 0. 0. 1. 0.]
	Inside Linear forward
	 output: [[2.4218897e-

When the number of latent dimensions equals 3, each latent variable is binary.

## Latent dimensions set to 4

In [13]:
AE = Autoencoder(latent_dims=4)
AE = AE.to(device)

# optimizer object
optimizer = torch.optim.Adam(params = AE.parameters(), lr = learning_rate)
#criterion = nn.CrossEntropyLoss()    # for target, does not accept a OHE vector
criterion = nn.NLLLoss()

trainAE(AE)


Epoch 1/2000	loss=-0.00215	Test loss: -0.13278
Epoch 201/2000	loss=-0.01177	Test loss: -0.72628
Epoch 401/2000	loss=-0.01486	Test loss: -0.91442
Epoch 601/2000	loss=-0.01601	Test loss: -0.98509
Epoch 801/2000	loss=-0.01618	Test loss: -0.99576
Epoch 1001/2000	loss=-0.01623	Test loss: -0.99864
Epoch 1201/2000	loss=-0.01624	Test loss: -0.99955
Epoch 1401/2000	loss=-0.01625	Test loss: -0.99985
Epoch 1601/2000	loss=-0.01625	Test loss: -0.99995
Epoch 1801/2000	loss=-0.01625	Test loss: -0.99999
Training finished!


In [14]:
print("Print prediction results:")
x_test = np.eye(num)[np.arange(num)]                        # Test data (one-hot encoded)
x_test = Variable(torch.from_numpy(x_test))
x_test = x_test.to(device)
#np.set_printoptions(2)
for x in x_test:
    print("\tInput: {} \t Output: {}".format(x.cpu().detach().numpy(), np.round(AE(features=x.float(),latent_dims=4).cpu().detach().numpy(),decimals=2)))

Print prediction results:
	Input: [1. 0. 0. 0. 0. 0. 0. 0.] 	 Output: [[1. 0. 0. 0. 0. 0. 0. 0.]]
	Input: [0. 1. 0. 0. 0. 0. 0. 0.] 	 Output: [[0. 1. 0. 0. 0. 0. 0. 0.]]
	Input: [0. 0. 1. 0. 0. 0. 0. 0.] 	 Output: [[0. 0. 1. 0. 0. 0. 0. 0.]]
	Input: [0. 0. 0. 1. 0. 0. 0. 0.] 	 Output: [[0. 0. 0. 1. 0. 0. 0. 0.]]
	Input: [0. 0. 0. 0. 1. 0. 0. 0.] 	 Output: [[0. 0. 0. 0. 1. 0. 0. 0.]]
	Input: [0. 0. 0. 0. 0. 1. 0. 0.] 	 Output: [[0. 0. 0. 0. 0. 1. 0. 0.]]
	Input: [0. 0. 0. 0. 0. 0. 1. 0.] 	 Output: [[0. 0. 0. 0. 0. 0. 1. 0.]]
	Input: [0. 0. 0. 0. 0. 0. 0. 1.] 	 Output: [[0. 0. 0. 0. 0. 0. 0. 1.]]


In [15]:
decoder_hook = AE.decoder_layer.register_forward_hook(inside_decoder)
for x in x_test:
    print('INPUT: {}'.format(x.cpu().detach().numpy()))
    out = AE(features=x.float(),latent_dims=4)
    #print(out)
decoder_hook.remove() #remove hook after use

INPUT: [1. 0. 0. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[0.0026432  0.04342017 0.9992932  0.9970497 ]]
	 output rounded to 2 decimals: [[0.   0.04 1.   1.  ]]
INPUT: [0. 1. 0. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[0.99950755 0.00742869 0.9990073  0.00167266]]
	 output rounded to 2 decimals: [[1.   0.01 1.   0.  ]]
INPUT: [0. 0. 1. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[0.99555737 0.99968433 0.28340858 0.00380852]]
	 output rounded to 2 decimals: [[1.   1.   0.28 0.  ]]
INPUT: [0. 0. 0. 1. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[0.995182   0.02445489 0.2369927  0.9997646 ]]
	 output rounded to 2 decimals: [[1.   0.02 0.24 1.  ]]
INPUT: [0. 0. 0. 0. 1. 0. 0. 0.]
	Inside Linear forward
	 output: [[9.9899703e-01 8.9323573e-04 5.9289695e-04 3.2633156e-04]]
	 output rounded to 2 decimals: [[1. 0. 0. 0.]]
INPUT: [0. 0. 0. 0. 0. 1. 0. 0.]
	Inside Linear forward
	 output: [[0.00378939 0.99801207 0.00874475 0.99312943]]
	 output rounded to 2 decimals: 

## Latent dimensions set to 2

In [16]:
AE = Autoencoder(latent_dims=2)
AE = AE.to(device)

# optimizer object
optimizer = torch.optim.Adam(params = AE.parameters(), lr = learning_rate)
#criterion = nn.CrossEntropyLoss()    # for target, does not accept a OHE vector
criterion = nn.NLLLoss()

trainAE(AE)


Epoch 1/2000	loss=-0.00204	Test loss: -0.12604
Epoch 201/2000	loss=-0.00661	Test loss: -0.40970
Epoch 401/2000	loss=-0.00841	Test loss: -0.51947
Epoch 601/2000	loss=-0.00973	Test loss: -0.59814
Epoch 801/2000	loss=-0.01119	Test loss: -0.69001
Epoch 1001/2000	loss=-0.01237	Test loss: -0.76305
Epoch 1201/2000	loss=-0.01376	Test loss: -0.84654
Epoch 1401/2000	loss=-0.01465	Test loss: -0.90204
Epoch 1601/2000	loss=-0.01522	Test loss: -0.93700
Epoch 1801/2000	loss=-0.01559	Test loss: -0.95936
Training finished!


In [17]:
print("Print prediction results:")
x_test = np.eye(num)[np.arange(num)]                        # Test data (one-hot encoded)
x_test = Variable(torch.from_numpy(x_test))
x_test = x_test.to(device)
#np.set_printoptions(2)
for x in x_test:
    print("\tInput: {} \t Output: {}".format(x.cpu().detach().numpy(), np.round(AE(features=x.float(),latent_dims=2).cpu().detach().numpy(),decimals=2)))

Print prediction results:
	Input: [1. 0. 0. 0. 0. 0. 0. 0.] 	 Output: [[0.98 0.   0.   0.01 0.   0.   0.01 0.  ]]
	Input: [0. 1. 0. 0. 0. 0. 0. 0.] 	 Output: [[0.   0.95 0.   0.   0.   0.02 0.02 0.02]]
	Input: [0. 0. 1. 0. 0. 0. 0. 0.] 	 Output: [[0.   0.   0.98 0.01 0.   0.   0.   0.01]]
	Input: [0. 0. 0. 1. 0. 0. 0. 0.] 	 Output: [[0.01 0.   0.01 0.98 0.   0.   0.   0.  ]]
	Input: [0. 0. 0. 0. 1. 0. 0. 0.] 	 Output: [[0.   0.   0.   0.   0.99 0.   0.01 0.  ]]
	Input: [0. 0. 0. 0. 0. 1. 0. 0.] 	 Output: [[0.   0.01 0.   0.   0.   0.99 0.   0.01]]
	Input: [0. 0. 0. 0. 0. 0. 1. 0.] 	 Output: [[0.01 0.03 0.   0.   0.01 0.   0.96 0.  ]]
	Input: [0. 0. 0. 0. 0. 0. 0. 1.] 	 Output: [[0.   0.02 0.01 0.   0.   0.01 0.   0.97]]


In [18]:
decoder_hook = AE.decoder_layer.register_forward_hook(inside_decoder)
for x in x_test:
    print('INPUT: {}'.format(x.cpu().detach().numpy()))
    out = AE(features=x.float(),latent_dims=2)
    #print(out)
decoder_hook.remove() #remove hook after use

INPUT: [1. 0. 0. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[1.5889947e-06 2.6691218e-06]]
	 output rounded to 2 decimals: [[0. 0.]]
INPUT: [0. 1. 0. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[0.59988695 0.43975922]]
	 output rounded to 2 decimals: [[0.6  0.44]]
INPUT: [0. 0. 1. 0. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[2.972657e-06 9.999975e-01]]
	 output rounded to 2 decimals: [[0. 1.]]
INPUT: [0. 0. 0. 1. 0. 0. 0. 0.]
	Inside Linear forward
	 output: [[1.9113144e-05 4.1637447e-01]]
	 output rounded to 2 decimals: [[0.   0.42]]
INPUT: [0. 0. 0. 0. 1. 0. 0. 0.]
	Inside Linear forward
	 output: [[9.9999630e-01 1.3771601e-05]]
	 output rounded to 2 decimals: [[1. 0.]]
INPUT: [0. 0. 0. 0. 0. 1. 0. 0.]
	Inside Linear forward
	 output: [[0.9999933  0.99887544]]
	 output rounded to 2 decimals: [[1. 1.]]
INPUT: [0. 0. 0. 0. 0. 0. 1. 0.]
	Inside Linear forward
	 output: [[4.1352379e-01 2.2040189e-05]]
	 output rounded to 2 decimals: [[0.41 0.  ]]
INPUT: [0. 0. 0. 0. 0.